### notebook to look at higher energy (> 1.6 MeV) events to see why beersheba gobbles them up and destroys them

We have pure signal data, so lets load that in

In [57]:
import sys,os,os.path
#sys.path.append("../../")   # cite IC from parent directory
sys.path.append("/gluster/data/next/software/IC_satkill/")
#sys.path.append("/gluster/data/next/software/IC_sophronia/")
#sys.path.append(os.path.expanduser('~/code/eol_hsrl_python'))
sys.path.append(os.path.expanduser('~/code/eol_hsrl_python'))
os.environ['ICTDIR']='/gluster/data/next/software/IC_satkill/'

import pandas as pd
import numpy  as np
import tables as tb

from IC.invisible_cities.evm.event_model                          import HitCollection

from IC.invisible_cities.dataflow                  import                  dataflow as  fl
from IC.invisible_cities.cities.components import track_blob_info_creator_extractor
from IC.invisible_cities.types  .ic_types          import         types_dict_tracks
from IC.invisible_cities.core                      import           system_of_units as units
from IC.invisible_cities.evm.event_model     import HitCollection
from IC.invisible_cities.evm.event_model     import Hit
from IC.invisible_cities.evm.event_model     import Cluster
from IC.invisible_cities.types.ic_types     import xy
from IC.invisible_cities.reco.paolina_functions import voxelize_hits, drop_end_point_voxels, make_track_graphs, get_track_energy

import IC.invisible_cities.core.core_functions                   as     coref
import IC.invisible_cities.io.dst_io                           as     dstio

from IC.invisible_cities.cities                 import beersheba as beerfun


from typing                import Dict
import time

from tqdm import tqdm

from fom_fitting import functions as func

In [10]:
# MC hits
data = pd.read_hdf('soph_df_data/Tl_signal_true_info.h5')
display(data)
# MC particles
data_2 = pd.read_hdf('soph_df_data/Tl_signal.h5')
display(data_2)
# sophronia hits
data_3 = pd.read_hdf('soph_df_data/Tl_signal_soph.h5')
display(data_3)

,event_id,particle_id,hit_id,x,y,z,time,energy,label
3236,8,52,0,315.166870,208.526917,731.868896,1.128019,0.000005,ACTIVE
3237,8,51,0,315.680695,208.035660,732.565308,0.961651,0.001719,ACTIVE
3238,8,51,1,316.120941,207.520065,733.248108,0.965438,0.001694,ACTIVE
3239,8,51,2,316.652252,206.863007,733.708008,0.969227,0.000674,ACTIVE
3240,8,51,3,316.177185,206.028381,733.763306,0.973017,0.004037,ACTIVE
...,...,...,...,...,...,...,...,...,...
100291,2900290,45,30,112.188820,95.269257,618.456299,1.479308,0.000619,ACTIVE
100292,2900290,45,31,112.198914,95.268547,618.448975,1.479575,0.001096,ACTIVE
100293,2900290,45,32,112.202179,95.277473,618.447754,1.479800,0.002352,ACTIVE
100294,2900290,45,33,112.203796,95.279572,618.450989,1.479932,0.001731,ACTIVE


,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,...,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
669,8,1,Tl208,True,0,397.326721,393.926666,531.145325,0.000000,397.326721,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,none,RadioactiveDecay
670,8,4,e-,False,1,397.326721,393.926666,531.145325,0.005979,392.497559,...,-0.274970,0.148090,-0.038526,-0.000000,-0.000000,0.000000,0.089120,5.964329e+00,RadioactiveDecay,eIoni
671,8,3,anti_nu_e,False,1,397.326721,393.926666,531.145325,0.005979,10972.416992,...,0.633006,-0.727434,0.716435,0.633006,-0.727434,0.716435,1.201306,2.006918e+04,RadioactiveDecay,Transportation
672,8,2,Pb208[3708.451],False,1,397.326721,393.926666,531.145325,0.005979,397.326721,...,-0.358036,0.579344,-0.677908,-0.000000,0.000000,-0.000000,0.000002,0.000000e+00,RadioactiveDecay,RadioactiveDecay
673,8,6,gamma,False,2,397.326721,393.926666,531.145325,0.005979,437.047546,...,0.292769,0.390412,-0.150731,0.000000,0.000000,-0.000000,0.510739,6.471526e+01,RadioactiveDecay,phot
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22897,2900290,44,e-,False,40,389.520874,394.557343,535.332153,0.025615,389.520874,...,-0.002189,-0.003671,0.005379,-0.000000,-0.000000,-0.000000,0.000046,1.169249e-07,compt,msc
22898,2900290,43,e-,False,40,389.520874,394.557343,535.332153,0.025615,389.520874,...,-0.000249,0.004221,-0.005044,-0.000000,0.000000,-0.000000,0.000042,1.120242e-07,compt,msc
22899,2900290,42,e-,False,40,389.520874,394.557343,535.332153,0.025615,389.520874,...,0.008071,-0.008042,-0.021972,0.000000,-0.000000,-0.000000,0.000599,8.628767e-06,compt,eIoni
22900,2900290,41,e-,False,40,389.520874,394.557343,535.332153,0.025615,389.396912,...,-0.704563,0.120819,1.098404,0.000000,-0.000000,-0.000000,0.895634,6.954247e-01,compt,eIoni


,event,time,npeak,Xpeak,Ypeak,nsipm,X,Y,Xrms,Yrms,Z,Q,E,Qc,Ec,track_id,Ep
4464,16,1.719597e+07,0,294.950939,190.401386,1,303.225,194.375,0.0,0.0,730.640203,6.666667,555.411165,-1.0,0.001900,-1,-1.0
4465,16,1.719597e+07,0,294.950939,190.401386,1,303.225,209.925,0.0,0.0,730.640203,5.797101,482.966206,-1.0,0.001653,-1,-1.0
4466,16,1.719597e+07,0,294.950939,190.401386,1,318.775,194.375,0.0,0.0,730.640203,9.159420,763.086640,-1.0,0.002611,-1,-1.0
4467,16,1.719597e+07,0,294.950939,190.401386,1,318.775,209.925,0.0,0.0,730.640203,7.014493,584.389133,-1.0,0.002000,-1,-1.0
4468,16,1.719597e+07,0,294.950939,190.401386,1,318.775,225.475,0.0,0.0,730.640203,5.913043,492.625569,-1.0,0.001694,-1,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127144,5800580,5.800581e+12,0,106.308278,101.721715,1,132.175,116.625,0.0,0.0,624.912488,7.826087,282.545951,-1.0,0.000906,-1,-1.0
127145,5800580,5.800581e+12,0,106.308278,101.721715,1,101.075,101.075,0.0,0.0,627.939966,6.840580,602.885083,-1.0,0.001922,-1,-1.0
127146,5800580,5.800581e+12,0,106.308278,101.721715,1,132.175,85.525,0.0,0.0,627.939966,4.000000,352.534483,-1.0,0.001122,-1,-1.0
127147,5800580,5.800581e+12,0,106.308278,101.721715,1,132.175,101.075,0.0,0.0,627.939966,4.869565,429.172396,-1.0,0.001376,-1,-1.0


In [31]:
# select events that have total energy > 2.0 MeV

MC_list = []

for event, df in data.groupby('event_id'):
    
    if (df.energy.sum()) > 2.0:
        MC_list.append(event*2)

        
soph_list = []

# crosscheck
for event, df in data_3.groupby('event'):
    if (df.Ec.sum()) > 2.0:
        soph_list.append(event)
        

if soph_list == MC_list:
    print("Identical lists, all good")
else:
    print("Non-identical, weirdness")
    print("{} non identical events out of {}/{} (MC/soph)".format(len(set(MC_list) ^ set(soph_list)), len(MC_list), len(soph_list)))

Non-identical, weirdness
78 non identical events out of 439/383 (MC/soph)


### Annoyingly, we need to track-ify the sophronia output, lets do that now

In [33]:
# the sophronia data
display(data_3)

,event,time,npeak,Xpeak,Ypeak,nsipm,X,Y,Xrms,Yrms,Z,Q,E,Qc,Ec,track_id,Ep
4464,16,1.719597e+07,0,294.950939,190.401386,1,303.225,194.375,0.0,0.0,730.640203,6.666667,555.411165,-1.0,0.001900,-1,-1.0
4465,16,1.719597e+07,0,294.950939,190.401386,1,303.225,209.925,0.0,0.0,730.640203,5.797101,482.966206,-1.0,0.001653,-1,-1.0
4466,16,1.719597e+07,0,294.950939,190.401386,1,318.775,194.375,0.0,0.0,730.640203,9.159420,763.086640,-1.0,0.002611,-1,-1.0
4467,16,1.719597e+07,0,294.950939,190.401386,1,318.775,209.925,0.0,0.0,730.640203,7.014493,584.389133,-1.0,0.002000,-1,-1.0
4468,16,1.719597e+07,0,294.950939,190.401386,1,318.775,225.475,0.0,0.0,730.640203,5.913043,492.625569,-1.0,0.001694,-1,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127144,5800580,5.800581e+12,0,106.308278,101.721715,1,132.175,116.625,0.0,0.0,624.912488,7.826087,282.545951,-1.0,0.000906,-1,-1.0
127145,5800580,5.800581e+12,0,106.308278,101.721715,1,101.075,101.075,0.0,0.0,627.939966,6.840580,602.885083,-1.0,0.001922,-1,-1.0
127146,5800580,5.800581e+12,0,106.308278,101.721715,1,132.175,85.525,0.0,0.0,627.939966,4.000000,352.534483,-1.0,0.001122,-1,-1.0
127147,5800580,5.800581e+12,0,106.308278,101.721715,1,132.175,101.075,0.0,0.0,627.939966,4.869565,429.172396,-1.0,0.001376,-1,-1.0


In [49]:
def create_row(evt, track_ID, track_E, track_no):
    '''
    create list row for filling df
    '''
    return [evt, track_ID, track_E, track_no]


def soph_track_info(df):
    '''
    Pass through soph hits dataframe and this will produce a new df with formatting:
                                        (MeV)                               (MeV)
    event_id   |   track_ID   |   track energy   | numb_of_tracks  | total energy
        0             0               0.57               2                1.11
        0             1               0.54               2                1.11
        1             0               1.65               1                1.65
        .             .                 .                .                  .
    '''
    
    # stuff needed for paolina track counting
    energy_threshold = 10
    min_voxels = 3
    
    base_vsize = 12 #mm
    
    data = []
    
    for (event, df_one) in df.groupby('event'):

        
        the_hits = []
        
        xs = df_one.X
        ys = df_one.Y
        zs = df_one.Z
        es = df_one.Ec
        
        for x, y, z, e in zip(xs, ys, zs, es):
            if np.isnan(e): continue
            h = Hit(0, Cluster(0, xy(x,y), xy(0,0), 0), z, e*1000, xy(0,0))
            the_hits.append(h)

        voxels = voxelize_hits(the_hits,
                        np.array([base_vsize, base_vsize, base_vsize]), False)

        
        (mod_voxels, dropped_voxels) = drop_end_point_voxels(voxels, energy_threshold, min_voxels)

        tracks = make_track_graphs(mod_voxels)
        
        track_number = len(tracks)
        
        for c, t in enumerate(tracks, 0):
            data.append(create_row(event, c, get_track_energy(t)/1000, track_number))
    
    
    return pd.DataFrame(data, columns = ['event_id', 'track_ID', 'track_energy', 'numb_of_tracks'])

In [ ]:
q_cut = 4
drop_dist = [16., 16.]

# Couple of functions used in beersheba.
cut_sensors       = beerfun.cut_over_Q   (q_cut, ['E', 'Ec'])
drop_sensors      = beerfun.drop_isolated(drop_dist, ['E', 'Ec'])

folder_path = '/gluster/data/next/files/TOPOLOGY_John/HYPPOS_DATA_QTHR/Q_THR4/PORT_1a/sophronia/'
file_names = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) and f.endswith('.h5')]


dfs = []
i = 0
tot = len(file_names)
for file in file_names:
    df_MC = dstio.load_dst(folder_path + file, 'RECO', 'Events')
    
    # remove background hits
    hits_cut = cut_sensors(df_MC.copy())
    hits_drop = drop_sensors(hits_cut.copy())
    
    dfs.append(soph_track_info(hits_drop))
    
    i += 1
    
    if (i%5 == 0):
        print("{}/{}".format(i,tot))
    
soph_hit_energy = pd.concat(dfs, axis = 0, ignore_index = True)

soph_hit_energy.to_hdf('soph_topology_info.h5', key = 'tracks', mode = 'w')

5/300
